In [2]:
import pandas as pd
import numpy as np
import re
import copy
import random

In [3]:
#open the file
cable_file = 'Diplomacy_W129.xlsx'
#cable_file = 'Missing.xlsx'
df = pd.read_excel(cable_file).T

In [4]:
class Cleaner:
    def __init__(self,cable_dataframe):
        self.df = cable_dataframe
        self.fd = self.df.T
        self.separate_rows()
        self.clean_all()
        self.cleaned = self.recompile()
        
    def separate_rows(self):
        self.turns = self.fd['turn']
        self.authors = self.fd['author civ']
        self.subjects = self.fd['subject civ']
        self.cables = self.fd['cable']
        try:
            self.categories = self.fd['Category']
            self.flags = self.fd['Flag']
            self.indx = self.df[0:1]
        except:
            pass
    
    def clean_all(self):
        self.clean_turn()
        self.clean_author()
        self.clean_subject()
        self.clean_cables()

    def clean_turn(self):
        pass
    def clean_author(self):
        #for i in range(len(self.authors)):
        for i in self.authors.keys():
            self.authors[i] = self.authors[i].replace('the','The')
    def clean_subject(self):
        #for i in range(len(self.subjects)):
        for i in self.subjects.keys():
            self.subjects[i] = self.subjects[i].replace('the','The')
    def clean_cables(self):
        bad_char = [',','!','.','"','\\',"'s'","'"]
        break_char = ['\r\n','\n','\r']
        #for i in range(len(self.cables)):
        for i in self.cables.keys():
            self.cables[i] = self.cables[i].replace('. ','\n')
            for j in bad_char:
                self.cables[i] = self.cables[i].replace(j,'')
            for j in break_char:
                self.cables[i] = self.cables[i].replace(j,'. ')
            self.cables[i] = self.cables[i].split('. ')
            

    def recompile(self):
        final = pd.DataFrame(self.turns).T
        final = final.append(self.authors)
        final = final.append(self.subjects)
        final = final.append(self.cables)
        try:
            final = final.append(self.categories)
            final = final.append(self.flags)
        except:
            pass
        return(final)

cdf = Cleaner(copy.copy(df))

In [70]:
cdf.cleaned

,40,242,338,367,419,522,546,853,913,1193,...,7326,7329,7416,7571,7583,7598,7663,7713,7855,7972
turn,11,28,36,38,42,49,50,67,72,87,...,444,444,448,462,462,463,466,470,481,491
author civ,Songhai,Morocco,Korea,Morocco,Germany,Morocco,Morocco,The Iroquois,Germany,The Iroquois,...,Germany,Germany,Korea,Germany,Korea,Germany,The Incas,Korea,Morocco,Songhai
subject civ,Morocco,The Iroquois,Germany,Germany,Songhai,The Incas,Korea,Songhai,Morocco,Germany,...,Korea,Korea,The Incas,Songhai,Morocco,Morocco,Germany,Songhai,Songhai,The Incas
cable,[The foreign office desires any intelligence p...,[The foreign office asks for any information o...,[The foreign office requires any knowledge on ...,[The foreign office requires any reporting in ...,[The foreign office asks for any information o...,[The foreign office requires any material avai...,[The foreign office requires any advice you ha...,[The foreign office requires any data in your ...,[The foreign office requires any advice obtain...,[The foreign office requires any data in your ...,...,[Urgent: we are now in an established state of...,[The foreign office desires any intelligence o...,[The foreign office asks for any news obtainab...,[The foreign office requires any information i...,[Welcome news: we have concluded violence with...,[The foreign office requires any data obtainab...,[Happy news: we have ceased violence with the ...,[ The foreign office desires any material obta...,[Insofar as we can assume the Songhai believes...,[The foreign office asks for any knowledge on ...
Category,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Flag,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,...,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing,Missing


In [28]:
class Dictionaries:
    def __init__(self,clean_object):
        self.cdf = clean_object
        self.define_signifiers()
        self.define_placenames()
        self.define_attributes()
        
        
        self.categorize_cables()
        
    def define_signifiers(self):
        self.signifiers = pd.DataFrame(columns = ['has_some','has_none']).T
        self.signifiers['Danger'] = [['risk','hazard','threat','problem','obstacle','danger'],['updates']]
        self.signifiers['Conquerability'] = [['conquering','for attack','dominating','taking over','target'],[]]
        self.signifiers['Economy'] = [['econom'],['any','foreign']]
        self.signifiers['Embassy'] = [['embassy','seat of government'],[]]
        self.signifiers['Greeting'] = [['wishes','reetin','honor','finds','I write','updates','alwyas'],['foreign']]
        self.signifiers['NA'] = [['no news'],[]]
        self.signifiers['Confidence'] = [['his assessment','in this','hypothesis','convinced','confid',
                                          'certain','very sure','sure source'],[]]
        self.signifiers['Info Request'] = [['would','any forecasting','Please','must have','wants any',
                                            'requests','needs any','asks for','desires any','requires any'],[]]
        self.signifiers['Military Strength'] = [['military'],['foreign','dominance ','military preeminence']]
        self.signifiers['War'] = [['war','conflict','the fighting','peace','news','Urgent','armistice'],
                                  ['foreign office','ward','preeminence','dominance','average','strong','poor',
                                   'weak','powerful','immense','pathetic']]
        self.signifiers['Trading'] = [['haggle','borders','trade','unit','barter','a deal','bargain','to deal'],
                                      ['embassy','seat of government','end the fighting']]
        self.signifiers['Prediction'] = [['preeminence','dominance'],[]]
        self.signifiers['Diplomatic Stance'] = [['Our relations with','protective','proceedings','policy','transactions','matters','guarded','hostile',
                                                 'aggressive','deceptive','afraid','friendly stance','neutral stance','friendly posture',
                                                 'neutral posture','likely neutral','seems neutral','looks neutral',
                                                 'friendly manner','neutral manner','friendly attitude','neutral attitude',
                                                 'feels neutral','feels friendly',
                                                 'now are neutral','are of late neutral','currently are neutral',
                                                 'these days are neutral','of late are neutral','at present are friendly',
                                                 'presently are neutral','at present neutral','are presently neutral',
                                                 'are currently neutral','are these days neutral','are now neutral',
                                                 'now are friendly','are of late friendly','currently are friendly',
                                                 'these days are friendly','of late are friendly','presently are friendly',
                                                 'at present friendly','are presently friendly','at present are neutral',
                                                 'are currently friendly','are these days friendly','are now friendly',
                                                 'likely friendly','seems friendly','looks friendly','appears neutral','appears friendly',
                                                 'protective position','afraid position','hostile position','friendly position',
                                                 'deceptive position','neutral position','guarded position',
                                                 'afraid disposition','hostile disposition','friendly disposition',
                                                 'deceptive disposition','neutral disposition','guarded disposition',
                                                 'protective disposition'],[' ally']]
        self.signifiers['Diplomatic Opinion'] = [['civilization','party','nation','country','open posture','popular posture','outward posture',
                                                  'reputed posture','professed posture','announced posture','a friend','A friend ','an ally',
                                                    'dealings','competitor','enemy','archnemesis','disposition','position'],
                                                 ['protective','afraid','hostile','friendly position','friendly disposition',
                                                  'guarded','deceptive','neutral position','neutral disposition','likely neutral','seems neutral',
                                                  'looks neutral','likely friendly','seems friendly','looks friendly','a friendly',
                                                 'appears neutral','appears friendly','feels friendly','feels neutral',
                                                  'appears friendly','appears neutral','neutral position','friendly position']]
 




    def define_placenames(self):
        self.countries = ['The Songhais','the Songhais','Songhai',
                         'The Germans','the Germans','Germany','German',
                         'The Koreans','the Koreans','Korean','Korea',
                         'The Moroccans','the Moroccans','Moroccan','Morocco',
                         'The Incas','the Incas','Incan','Inca',
                         'The Iroquois','the Iroquois','Iroquois']
        self.city_states = ['Bogota', 'Budapest', 'Florence', 'Ife', 'Kathmandu', 'Kyzyl',
                            'Mombasa', 'Panama City', 'Singapore', 'Yerevan']
    
    def define_attributes(self):
        self.data = self.cdf.cleaned
        self.turns = self.cdf.turns
        self.authors = self.cdf.authors
        self.subjects = self.cdf.subjects
        self.cables = self.cdf.cables
    
    def categorize_cables(self):
        data = copy.copy(self.data)
        cols = list(data.index)
        if len(cols) < 5:
            cols.extend(['Category','Flag'])
        self.cat_cab = pd.DataFrame(columns = cols).T
        for i in data:
            if len(data[i]['cable'])>1:
                for j in data[i]['cable'][2:len(data[i]['cable'])-3]:
                    if len(j)>2:
                        cat,flag = self.get_cat(j)
                        self.cat_cab[len(self.cat_cab.T)] = [data[i]['turn'],data[i]['author civ'],data[i]['subject civ'],j,cat,flag]
                        #print([data[i]['turn'],data[i]['author civ'],data[i]['subject civ'],j,cat,flag])
            else:
                j = data[i]['cable'][0]
                cat,flag = self.get_cat(j)
                self.cat_cab[i] = [data[i]['turn'],data[i]['author civ'],data[i]['subject civ'],j,cat,flag]
                #print([data[i]['turn'],data[i]['author civ'],data[i]['subject civ'],j,cat,flag])
    
    def get_cat(self,cable):
        cab = cable
        cat = []
        for i in self.signifiers:
            for j in self.signifiers[i][0]:
                if j in cab:
                    cat.append(i)
            acab = np.array(cat)
            cat = list(np.unique(acab))
            for j in self.signifiers[i][1]:
                if j in cab:
                    if i in cat:
                        cat.remove(i)
        if len(cat) == 0:
            flag = 'Missing'
            cat = ''
        elif len(cat) > 1:
            print(cat)
            flag = 'Multiple'
            cat = cat[-1]
            #cat = ' AND '.join(cat)
        else:
            flag = 'None'
            cat = cat[0]
        return(cat,flag)
    
    
    def generalize_cables(self):      
        pass
        
dic = Dictionaries(copy.copy(cdf))
print('done')


['Diplomatic Opinion', 'Economy']
['Diplomatic Opinion', 'Diplomatic Stance']
['Diplomatic Opinion', 'Diplomatic Stance']
['Diplomatic Opinion', 'Diplomatic Stance']
['Diplomatic Opinion', 'Diplomatic Stance']
['Diplomatic Opinion', 'Economy']
['Diplomatic Opinion', 'Diplomatic Stance']
['Diplomatic Opinion', 'Military Strength']
['Diplomatic Opinion', 'Military Strength']
['Diplomatic Opinion', 'Diplomatic Stance']
['Diplomatic Opinion', 'Diplomatic Stance']
['Diplomatic Opinion', 'Military Strength']
['Diplomatic Opinion', 'Military Strength']
['Diplomatic Opinion', 'Diplomatic Stance']
['Diplomatic Opinion', 'Diplomatic Stance']
['Diplomatic Opinion', 'Economy']
['Diplomatic Opinion', 'Economy']
['Diplomatic Opinion', 'Military Strength']
['Diplomatic Opinion', 'Diplomatic Stance']
['Conquerability', 'Info Request', 'Military Strength']
['Diplomatic Opinion', 'Diplomatic Stance']
['Diplomatic Opinion', 'Diplomatic Stance']
['Diplomatic Opinion', 'Diplomatic Stance']
['Diplomatic Opi

In [31]:
a = dic.cat_cab.T

print(a.Category.unique())
cats = a.Category.unique()

for i in cats:
    b = a[a.Category == i]
    b.to_excel('categoried/'+i+'.xlsx')

['Military Strength' 'Diplomatic Stance' 'Conquerability'
 'Diplomatic Opinion' 'Economy' 'Prediction' 'Confidence' 'Danger'
 'Info Request' 'Embassy' 'Trading' 'War']


In [29]:
a = dic.cat_cab.T
a[a.Flag == 'Multiple'][a.Category == 'Diplomatic Opinion AND Diplomatic Stance']

C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,turn,author civ,subject civ,cable,Category,Flag


In [ ]:


# turn,sender,receiver,Civ 1,Civ 2,stat,value,original cable,


